## Design of Algorithms by Induction

These notes are based on the book *Introduction to Algorithms: A Creative Approach* (chapter 5).

    It is not necessary to design the steps required to solve a problem from scratch; it suffices to guarantee that 
    (1) it is possible to solve a small instance of the problem (the base case), and 
    (2) a solution to every problem can be constructed from solutions of smaller subproblems (the inductive step).

<br/><center><img src=img/design_principle.png></center>

### Evaluating Polynomials

**Problem.** Given a sequence of real numbers $(a_n,a_{n-1},...,a_0)$, and a real number $x$, compute the value of the polynomial $$P_n(x)=a_nx^n+a_{n-1}x^{n-1}+...+a_1x+a_0\text{.}$$

Reduce the problem by removing $a_n$, remaining $$P_{n-1}(x)=a_{n-1}x^{n-1}+a_{n-1}x^{n-2}+...+a_1x+a_0\text{.}$$ Thus, we are lead to the following hypothesis:

**Induction Hypothesis:** we know how to evaluate $P_{n-1}(x)$.

To solve the base case, compute $a_0$, which is trivial. To solve the originial problem (computing $P_n(x)$):
1. compute $x^n$, 
2. multiply it by $a_n$,
3. add the result to $P_{n-1}(x)$.

That is, $P_n(x)=a_nx^n+P_{n-1}(x)$.

In [1]:
def evaluate(A, x):
    n = len(A) - 1
    if n == 0:
        return A[0]
    else:
        return A[n] * x**n + evaluate(A[:n], x) # A[:n] is equivalent to A[0..n-1]

**Observation.** Including exponentiation as a basic operation, it requires constant time to compute. Otherwise, we would have to strengthen the induction hypothesis including *we also know how to compute $x^{n-1}$*.

**Complexity.** The running time can be described by the recurrence
$$
T(n)=
\left\{
\begin{array}{ll}
      O(1)          & n=1  \\
      T(n-1) + O(1) & n > 1
\end{array} 
\right.
$$
which is $O(n)$.

A simpler algorithm can be designed removing $a_n$ *and* $a_0$, which is $$P'_{n-1}(x)=a_nx^{n-1}+a_{n_1}^{n-2}+...+a_1\text{.}$$
Notice that $a_n$ is now the ($n-1$)th coefficient, $a_{n-1}$ is the ($n-2$)th coefficient, and so on.

**Induction Hypothesis (reversed order):** we know how to evaluate $P'_{n-1}(x)$.

The base case remaings the same, but this hypothesis is more suited to our purposes because it's easier to extend. Thus, to solve $P(x)$:
1. multiply $x$ by $P'_{n-1}(x)$,
2. add it to $a_0$.

This algorithm is known as **Horner's rule**.

In [2]:
def horner_rule(A, x):
    n = len(A) - 1
    result = A[n]
    for i in range(n):
        result = x * result + A[i]
    return result

**Complexity.** Line 2 and 3 are $O(1)$ and the loop is $O(n)$, therefore the running time is $O(n)$.

### The Celebrity Problem

**Problem.** Given a $n \times n$ adjacency matrix, determine whether there exists an $i$ such that all the entries in the $i$th column are 1 (except for the $ii$th entry), and all the entries in the $i$th row are 0 (except for the $ii$th entry).

**Definition.** Among $n$ persons, a *celebrity* is someone who is known by everyone but does not know anyone.

The problem is to find the celebrity, if one exists, by asking if a person $A$ knows another person $B$. The goal is to minimize the number of questions, because since there are $n(n-1)/2$ pairs of persons, we may need to ask $n(n-1)$ questions in the worst case, what would require an $O(n^2)$ algorithm.

Using a graph-theoretical formulation, we can build a directed graph with the vertices corresponding to the persons and an edge from person $A$ to person $B$ if $A$ knows $B$. A celebrity is a vertex with indegree $n-1$ and outdegree 0, also known as a *sink*.

**Definition.** A *sink* is a vertex with indegree $n-1$ and outdegree $0$.

Notice that a graph can have at most one sink.

The input to the problem corresponds to a $n \times n$ adjacency matrix whose $ij$ entry is 1 if $i$th person knows $j$th person, and 0 otherwie.

In [3]:
M = [
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,0], # 2
    [0,0,0,0]  # 3
]

**Induction Hypothesis:** we know how to find the celebrity among the first $n-1$ persons.

The base case is simple: if there is only one person, she is the celebrity. 

Since there is at most one celebrity, there are three possibilities:
1. the celebrity is among the first $n-1$ persons,
2. the celebrity is the $n$th person,
3. there is no celebrity.

To handle case 1, just check if the $n$th person knows the celebrity, and that the celebrity does not know the $n$th person. To handle the other two cases, we may need to ask $2(n-1)$ questions to determine if the $n$th person is the celebrity (2 questions per vertex minus 2 for the $n$th one). This may require $n(n-1)$ questions in the worst case. Not good.

Instead, let's identify someone as a noncelebrity so we can reduce the size of the problem from $n$ to $n-1$ eliminating someone from consideration (we have much more noncelebrities after all). This can be done like so:
```
if A knows B then
    A is not a celebrity
else
    B is not a celebrity 
```

Consider again the three cases. Using the idea above, we can eliminate one person and solve the problem for the other $n-1$ persons (by our hypothesis).
- Case 1: with two more questions, we verify this a celebrity for the whole set.
- Case 2: guaranteed to not occur since we are eliminating noncelebrities, thus the $n$th person cannot be the celebrity. 
- Case 3: there is no celebrity among the $n$ persons.

In [4]:
def find_celebrity(Know):
    n = len(Know) - 1
    i, j, next = 0, 1, 2
    
    # eliminate all but one candidate, 
    # i.e someone everyone knows.
    while next <= n + 1:
        if Know[i][j]:                                 # i is not celebrity
            i = next
        else:                                          # j is not celebrity
            j = next
        next += 1
    if i == n + 1:
        candidate = j
    else:
        candidate = i
    
    # check that the candidate is indeed
    # the celebrity.
    wrong = False
    k = 0
    while k <= n and not wrong:
        if Know[candidate][k]:                         # if candidate knows someone
            wrong = True
        if not Know[k][candidate] and candidate != k:  # if someone does not know the candidate
            wrong = True
        k += 1
    if not wrong:
        print(f'The celebrity is {candidate}')
    else:
        print('There is no celebrity')

**Complexity.** We compute at most $O(n)$ "questions" for the whole set of persons. 

In [5]:
find_celebrity([
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,1], # 2
    [0,0,0,0]  # 3
])

find_celebrity([
   # 0 1 2 3
    [0,0,0,1], # 0
    [0,0,1,1], # 1
    [0,1,0,0], # 2
    [0,0,0,0]  # 3
])

The celebrity is 3
There is no celebrity


### Computing Balance Factors in Binary Trees

**Problem.** Given a binary tree $T$ with $n$ nodes, compute the balance factors of all the nodes.

First of all, let's define an ADT for our binary tree along with a function to traverse it, what is height, and what is a balance factor.

In [26]:
class BinaryTree:
    def __init__(self, root_value):
        self.left = None
        self.right = None
        self.height = 0
        self.balance_factor = 0
        self.value = root_value
    def __repr__(self):
        return f'<{self.value} (h={self.height}/bf={self.balance_factor})>'
    def insert(self, value):
        if value < self.value:
            if self.left:
                self.left.insert(value)
            else:
                self.left = BinaryTree(value)
        elif value > self.value:
            if self.right:
                self.right.insert(value)
            else:
                self.right = BinaryTree(value)
        else:
            raise ValueError(f'{value} is already inserted')
      
def traverse_postorder(T):
    if T:
        traverse_postorder(T.left)
        traverse_postorder(T.right)
        print(T)

**Definition.** The *height* of a node $v$ is the number of edges between $v$ and the farthest leaf down the tree.

**Definition.** The *balance factor* of a node *v* is the difference between the height of the node's left subtree and the height of the node's right subtree.

**Induction Hypothesis:** we know how to compute the balance factors of all nodes in trees that have less than $n$ nodes.

The base case of $n=1$ is trivial (the balance factor is 0). Given a tree with $n>1$ nodes, we could use the hypothesis to compute the balance factors of the root's two subtrees. But we are left with the root. We can't compute its balance factor because it doesn't depend on the balance factors of its children, but on their height! Hence, we need to strengthen the induction hypothesis:

**Stronger Induction Hypothesis:** we know how to compute balance factors **and heights** of all nodes in trees that have less than $n$ nodes.

The base case remains the same. Now, we can easily compute the root's balance factors by calculating the difference between the heights of its two subtrees. Furthermore, we can also compute its height - the maximal height of the its subtrees plus 1.

In [27]:
def height_and_bf(T):
    if not T:
        return 0, 0
    else:
        left_bf, left_height = height_and_bf(T.left)
        right_bf, right_height = height_and_bf(T.right)
        balance_factor = left_height - right_height
        height = max(left_height, right_height) + 1       # add 1 to compute BF and return this value
        T.balance_factor = balance_factor
        T.height = height - 1                             # subtract 1 so every node has correct height
        return balance_factor, height

Notice we had to follow the induction hypothesis precisely! We added one more assumption (computing height), so we must adjust the algorithm to compute the height and balance factors *separately*.

In [31]:
mytree = BinaryTree(6)
mytree.insert(2)
mytree.insert(7)
mytree.insert(1)
mytree.insert(4)
mytree.insert(8)

height_and_bf(mytree)

traverse_postorder(mytree)

<1 (h=0/bf=0)>
<4 (h=0/bf=0)>
<2 (h=1/bf=0)>
<8 (h=0/bf=0)>
<7 (h=1/bf=-1)>
<6 (h=2/bf=0)>


### Finding the Maximum Consecutive Subsequence

**Problem.** Given a sequence $x_1,x_2,...,x_n$ of real numbers (not necessarily posivite), find a subsequence $x_i,x_{i+1},...,x_j$ (of consecutive elements) such that the sum of the numbers in it is maximum over all subsequences of consecutive elements.

For example, take the sequence $(2,-3,1.5,-1,3,-2,-3,3)$. Its maximum consecutive subsequence (MCS) is $(1.5,-1,3)$, which sums up to $3.5$. There may be several MCS in a given sequence. If all numbers are negative, then the MCS is empty (by definition, its sum is $0$). We would like to have an algorithm that solves the problem and reads the sequence in order *only once*.

**Induction Hypothesis:** we know how to find the maximum consecutive subsequence in sequences of size less than $n$.

If $n=1$, the MCS consists of the single number if that number is positive, or the empty subsequence otherwise. Now, consider a sequence $S=(x_1,x_2,...,x_n)$ of size $n>1$. By induction we know how to find a MCS in $S'=(x_1,x_2,...,x_{n-1})$. If all numbers in $S'$ are negative, then its MCS is empty and we consider only $x_n$. If there is a MCS $(x_i,x_{i+1},...,x_j)$ in $S'$ for $1 \leq i,j \leq n-1$, we have two cases:

a) If $j=n-1$, then it is easy to extend the solution to $S$: just append $x_n$ at the end such that we have $(x_i,x_{i+1},...,x_j,x_n)$.

b) If $j<n-1$, then either $(x_i,x_{i+1},...,x_j)$ remains maximum or there is another MCS in $S$ when we add $x_n$.

In [25]:
def maxsub(S):
    n = len(S) - 1
    if n == 0:
        if S[n] < 0:
            return []
        else:
            return [S[n]]
    else:
        subsequence = maxsub(S[:n])
        solution = subsequence + [S[n]]
        if sum(subsequence) < sum(solution):
            return solution
        else:
            return subsequence

In [26]:
S = [0,-1,3,-2,1] # should be [3]
maxsub(S)

[0, 3, 1]

Notice that the implementation above doesn't consider case (b) when there is another MCS afetr we add `S[n]`. Actually, it correctly computes the maximum subsequence, but it may not be consecutive! Hence, the key is to strengthen the induction hypothesis. The problem we had with the straightforward induction hypothesis was that **$x_n$ may extend a subsequence that is not maximum in $S'$**, and thus create a new maximum subsequence. Knowing only the MCS in $S'$ is thus not sufficient.

We must extend $S'$ with $x_n$ only when it ends at $n-1$. That is, a suffix of $S'$.
 
**Stronger Induction Hypothesis:** we know how find, in sequences of size less than $n$, the maximum consecutive subsequence overall and also the maximum consecutive subsequence that it a suffix.

### Dynamic Programming: The Knapsack Problem